In [3]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer

bptt = 35

In [11]:
sz = bptt
mask = (torch.triu(torch.ones(sz, sz))) # 원래 triu 는 tri matrix생성함수로 upper tri matrix생성 대각선부터 그 위로만
print(mask)
print(mask.shape)

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [0., 1., 1.,  ..., 1., 1., 1.],
        [0., 0., 1.,  ..., 1., 1., 1.],
        ...,
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 0., 1., 1.],
        [0., 0., 0.,  ..., 0., 0., 1.]])
torch.Size([35, 35])


In [12]:
mask = (torch.triu(torch.ones(sz, sz)) == 1) # 여기에 bool을 쓰고 싶어서 변경
print(mask)
print(mask.shape)

tensor([[ True,  True,  True,  ...,  True,  True,  True],
        [False,  True,  True,  ...,  True,  True,  True],
        [False, False,  True,  ...,  True,  True,  True],
        ...,
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ..., False,  True,  True],
        [False, False, False,  ..., False, False,  True]])
torch.Size([35, 35])


In [13]:
mask = (torch.triu(torch.zeros(sz, sz))) # 참고로 zeros를 넣으면 upper에도 0이 들어가서 맛이감
print(mask)
print(mask.shape)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([35, 35])


In [14]:
mask = (torch.triu(torch.ones(sz, sz) * 5)) # 이렇게 다른 값을 넣을수도 있음
print(mask)
print(mask.shape)

tensor([[5., 5., 5.,  ..., 5., 5., 5.],
        [0., 5., 5.,  ..., 5., 5., 5.],
        [0., 0., 5.,  ..., 5., 5., 5.],
        ...,
        [0., 0., 0.,  ..., 5., 5., 5.],
        [0., 0., 0.,  ..., 0., 5., 5.],
        [0., 0., 0.,  ..., 0., 0., 5.]])
torch.Size([35, 35])


In [15]:
mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1) # 여기서는 lower를 원하는지 transpose를 해줌
print(mask)
print(mask.shape)

tensor([[ True, False, False,  ..., False, False, False],
        [ True,  True, False,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False],
        ...,
        [ True,  True,  True,  ...,  True, False, False],
        [ True,  True,  True,  ...,  True,  True, False],
        [ True,  True,  True,  ...,  True,  True,  True]])
torch.Size([35, 35])


In [21]:
mask = (torch.tril(torch.ones(sz, sz)) == 1) # 근데 그냥 애초에 tril 즉 tri low로 하면됨
print(mask)
print(mask.shape)

tensor([[ True, False, False,  ..., False, False, False],
        [ True,  True, False,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False],
        ...,
        [ True,  True,  True,  ...,  True, False, False],
        [ True,  True,  True,  ...,  True,  True, False],
        [ True,  True,  True,  ...,  True,  True,  True]])
torch.Size([35, 35])


In [22]:
mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0)) 
# 대충 딱봐도 mask를 다시 float으로 바꾸고 masked_fill함수로 mask가 0이면 -inf를 1이면 0을 넣어주는데 이거 아무리봐도 이상함
print(mask)
print(mask.shape)

tensor([[0., -inf, -inf,  ..., -inf, -inf, -inf],
        [0., 0., -inf,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        ...,
        [0., 0., 0.,  ..., 0., -inf, -inf],
        [0., 0., 0.,  ..., 0., 0., -inf],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([35, 35])


In [19]:
mask = (torch.tril(torch.ones(sz, sz))) # 일단 굳이 ==1같은거로 bool로 갈필요도 없고, transpose도 괜히함
mask = mask.masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0)) 
print(mask)
print(mask.shape)

tensor([[0., -inf, -inf,  ..., -inf, -inf, -inf],
        [0., 0., -inf,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        ...,
        [0., 0., 0.,  ..., 0., -inf, -inf],
        [0., 0., 0.,  ..., 0., 0., -inf],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([35, 35])


In [20]:
# 아래꺼가 원본
mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
print(mask)
print(mask.shape)

tensor([[0., -inf, -inf,  ..., -inf, -inf, -inf],
        [0., 0., -inf,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        ...,
        [0., 0., 0.,  ..., 0., -inf, -inf],
        [0., 0., 0.,  ..., 0., 0., -inf],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([35, 35])
